# Finetuning GPT 3.5 Turbo

This notebook finetunes GTP 3.5 Turbo


# Step 0: Setup

In [ ]:
# upgrade openai library, restart kernel

#pip install --upgrade openai
#pip install requests


In [27]:
# imports and settings
# TODO: enter  API key and org ID from your ChatGPT Plus account, see here: https://platform.openai.com/account/org-settings

import os
import openai
import requests
import json
import datetime

OPENAI_API_KEY = "YOURAPIKEY" # TODO: Enter API key
ORG_ID = "YOUORGID" # TODO: Enter Org ID

TRAINING_FILE_NAME = "data-finetuning-10examples.jsonl" # training file in JSONL format

# set API key
#openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


'\nHEADERS = {\n    "Authorization": f"Bearer {OPENAI_API_KEY}",\n    "Content-Type": "application/json"\n}\n'

# Step 1: Prepare training data for finetuning

## Requirements:
min. 10 examples needed for training, format: JSONL

## Hint: Generate dummy data with ChatGPT (Prompt)

Erstelle Dummy-Daten, die zum Model-Finetuning von OpenAIs GPT-3 Turbo verwendet werden können.
Die Dummy-Daten sollen den Kundensupport für einen Hochleistungs-Staubsauger mit Namen "T-1000" simulieren. 
Denke Dir mögliche Kundenfragen und die dazu passende Kundenservice-Antwort aus.
Der Kunde soll genervt anfragen. 
Der Kundenservice soll höflich und kurz antworten. 
Verwende das im folgenden gegebene JSONL-Format.

Generiere den System-Prompt und 10 Beispiele.

Beispiel für das JSONL-Format:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

In [88]:

system_prompt = "Du chattest jetzt mit dem Kundenservice für den Hochleistungs-Staubsauger T-1000."

data = [
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Warum saugt mein T-1000 nicht mehr richtig?"}, 
                      {"role": "assistant", "content": "Es könnte ein verstopfter Filter sein. Bitte überprüfen Sie den Filter und reinigen Sie ihn gegebenenfalls."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Warum ist dieser Staubsauger so laut? Ich kann kaum denken, wenn er läuft!"}, 
                      {"role": "assistant", "content": "Entschuldigung für die Unannehmlichkeiten. Die Lautstärke kann aufgrund von Fremdkörpern in der Bürste steigen. Bitte überprüfen Sie diese."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Warum hält der Akku nicht so lange, wie in der Werbung versprochen?"}, 
                      {"role": "assistant", "content": "Es könnte viele Gründe geben. Zum Beispiel könnte ein häufiges und langes Saugen den Akku schneller entladen. Stellen Sie sicher, dass Sie den Staubsauger nach jedem Gebrauch aufladen."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Mein T-1000 verliert ständig an Saugkraft, was soll das?"}, 
                      {"role": "assistant", "content": "Möglicherweise ist der Behälter voll. Bitte leeren Sie ihn und versuchen Sie es erneut."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Wieso ist das Kabel meines Staubsaugers ständig verheddert?"}, 
                      {"role": "assistant", "content": "Stellen Sie sicher, dass Sie das Kabel korrekt aufwickeln und Lagerungstipps befolgen."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Warum funktioniert die Fernbedienung nicht mehr?"}, 
                      {"role": "assistant", "content": "Es könnten die Batterien leer sein. Bitte tauschen Sie diese aus."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Der Filter war nach nur einem Monat schon kaputt. Was soll das?"}, 
                      {"role": "assistant", "content": "Das tut uns leid. Bitte wenden Sie sich an unseren Support, um einen Ersatz zu erhalten."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Kann ich kein normales Reinigungsmittel für meinen T-1000 verwenden?"}, 
                      {"role": "assistant", "content": "Wir empfehlen spezielle Reinigungsmittel, um die Lebensdauer zu verlängern und Beschädigungen zu vermeiden."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Wo finde ich Ersatzteile für meinen T-1000?"}, 
                      {"role": "assistant", "content": "Ersatzteile können direkt über unsere Website oder autorisierte Händler bezogen werden."}]},
    {"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Wie reinige ich die Bürsten meines T-1000 richtig?"}, 
                      {"role": "assistant", "content": "Entnehmen Sie die Bürsten und entfernen Sie Haare und Schmutz. Eine detaillierte Anleitung finden Sie im Benutzerhandbuch."}]}
]

# Save data to a file in JSONL format
with open(TRAINING_FILE_NAME, 'w') as file:
    for item in data:
        file.write(json.dumps(item) + '\n')

"\n# Save data to a file in JSONL format\nwith open(TRAINING_FILE_NAME, 'w') as file:\n    for item in data:\n        file.write(json.dumps(item) + '\n')\n"

# Step 2: Upload training file


In [39]:
response_upload = openai.File.create(
  file=open(TRAINING_FILE_NAME, "rb"),
  purpose='fine-tune'
)

TRAINING_FILE_ID = response_upload.id

# print upload response to check if successful
response_upload

<File file id=file-Hxy2AYUUFwXXUgU1h6aSYqI0 at 0x1157f8a70> JSON: {
  "object": "file",
  "id": "file-Hxy2AYUUFwXXUgU1h6aSYqI0",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 3858,
  "created_at": 1692904623,
  "status": "uploaded",
  "status_details": null
}

In [83]:
# see uploaded files, delete unused files if necessary

openai.File.list()
#openai.File.delete("file-yNehrD1NPaov1ecrjJUaV3Zx")


<OpenAIObject list at 0x1159c3fb0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-Hxy2AYUUFwXXUgU1h6aSYqI0",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 3858,
      "created_at": 1692904623,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-5oird9a7oX4UhX27mMx9AZBE",
      "purpose": "fine-tune-results",
      "filename": "step_metrics.csv",
      "bytes": 1746,
      "created_at": 1692905471,
      "status": "processed",
      "status_details": null
    }
  ]
}

# Step 3: Start finetuning training job

In [40]:
job = openai.FineTuningJob.create(training_file=TRAINING_FILE_ID, model="gpt-3.5-turbo-0613")
job

<FineTuningJob fine_tuning.job id=ftjob-EZ5afGpt3H6YJDxIPDm1XMKt at 0x1157cbf50> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-EZ5afGpt3H6YJDxIPDm1XMKt",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692904759,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-UHxyBndSEttpmWmDYb3DDxzn",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-Hxy2AYUUFwXXUgU1h6aSYqI0",
  "hyperparameters": {
    "n_epochs": 10
  },
  "trained_tokens": null
}

### List fine-tuning jobs and show status

In [73]:
# List fine-tuning jobs and show status

jobs = openai.FineTuningJob.list(limit=10)

for job in jobs.data:
    print("*** job-id: " + job.id + " ***")
    print("Status: " + openai.FineTuningJob.retrieve(job.id).status )

    job_events = openai.FineTuningJob.list_events(id=job.id, limit=20)
    for e in job_events.data:
        timestamp = formatted_date = datetime.datetime.fromtimestamp(e["created_at"]).strftime('%H:%M:%S')
        print(timestamp + " - " + e.message)


*** job-id: ftjob-EZ5afGpt3H6YJDxIPDm1XMKt ***
Status: succeeded
21:31:13 - Fine-tuning job successfully completed
21:31:10 - New fine-tuned model created: ft:gpt-3.5-turbo-0613:mediaworx-berlin-ag::7rAKrrlF
21:31:06 - Step 100/100: training loss=0.01
21:30:54 - Step 90/100: training loss=0.01
21:30:44 - Step 80/100: training loss=0.02
21:30:32 - Step 70/100: training loss=0.07
21:30:22 - Step 60/100: training loss=0.08
21:30:10 - Step 50/100: training loss=0.15
21:29:57 - Step 40/100: training loss=0.18
21:29:47 - Step 30/100: training loss=0.45
21:29:35 - Step 20/100: training loss=0.39
21:29:23 - Step 10/100: training loss=1.50
21:19:20 - Fine tuning job started
21:19:19 - Created fine-tune: ftjob-EZ5afGpt3H6YJDxIPDm1XMKt


### Check if your finetuned model is visible


In [82]:
# check if your finetuned model is visible

sorted_models = sorted(models.data, key=lambda m: m.id)

for m in sorted_models:
    print("model: " + m.id)

model: ada
model: ada-code-search-code
model: ada-code-search-text
model: ada-search-document
model: ada-search-query
model: ada-similarity
model: babbage
model: babbage-002
model: babbage-code-search-code
model: babbage-code-search-text
model: babbage-search-document
model: babbage-search-query
model: babbage-similarity
model: code-davinci-edit-001
model: code-search-ada-code-001
model: code-search-ada-text-001
model: code-search-babbage-code-001
model: code-search-babbage-text-001
model: curie
model: curie-instruct-beta
model: curie-search-document
model: curie-search-query
model: curie-similarity
model: davinci
model: davinci-002
model: davinci-instruct-beta
model: davinci-search-document
model: davinci-search-query
model: davinci-similarity
model: ft:gpt-3.5-turbo-0613:mediaworx-berlin-ag::7rAKrrlF
model: gpt-3.5-turbo
model: gpt-3.5-turbo-0301
model: gpt-3.5-turbo-0613
model: gpt-3.5-turbo-16k
model: gpt-3.5-turbo-16k-0613
model: text-ada-001
model: text-babbage-001
model: text-cu

# Step 4: Query your finetuned model

In [86]:

MODEL = "ft:gpt-3.5-turbo-0613:mediaworx-berlin-ag::7rAKrrlF" # TODO: Enter your model name

completion = openai.ChatCompletion.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "Du chattest jetzt mit dem Kundenservice für den Hochleistungs-Staubsauger T-1000."},
    {"role": "user", "content": "Ich will meinen T-1000 mit Pril reinigen. Geht das?"}
  ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Wir empfehlen nur spezielle Reinigungsmittel, um Besch\u00e4digungen zu vermeiden."
}
